In [ ]:
__author__="Dominic A. Sirianni"
__credits__=["Asim Alenaizan", "Daniel L. Cheney", "C. David Sherrill"]

__copyright__="(c) 2018, The Sherrill Group"
__license__="BSD-3-Clause"
__date__="2018-01-23"

# Molecular Alignment via the Kabsch Algorithm

In order to effectively compare molecular structures via the root-mean-squared deviation (RMSD) metric, one must first place the structures in so-called "maximal coincidence," i.e., aligned to one another such that the overlap of the structures is maximized.  This notebook will provide working implementations of the methods employed in the accompanying [paper]() and [supplementary information]() for achieving this maximal coincidence for molecular structures, as well as some discussion of these implementations' details.

### Alignment Strategy

Consider two sets of ordered molecular coordinates $\mathbb{P},\,\mathbb{Q}$ of a molecular system $\mathcal{M}$, collected into matrices $P$ and $Q$, respectively.  We wish to transform $\mathbb{P}$ and $\mathbb{Q}$ such that without perturbing the geometries themselves, these sets can be placed into maximal alignment.  To do this, we must apply transformations which do not scale or otherwise distort these geometries; thus, we are limited to translations and rotations.  

By first translating both $P$ and $Q$ to their respective centroids (the geometric average of atomic positions), the problem of optimal alignment can be reduced to finding the rotation which maximally aligns the coordinates, since this translation is the optimal one.  

### Executing notebook cells

Jupyter notebooks conveniently separate code into "cells," which can be executed separately and repeatedly by selecting the cell with your cursor and typing `Shift+Enter`.  Note that to execute the cells labeled "Example 0", "Example 1", and "Example 2", you must execute the cell below to import relevant packages and define constants &  functions.

### Notebook contents

This notebook includes several functions which can be used to obtain optimal molecular alignment, in the cell below.  For documentation of these functions as well as assistance using them, type `help(function_name)` in a cell after executing the cell below.

In [ ]:
# ==> Import packages, define functions & constants <==
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# Constants
bohr2angstroms = 0.52917720859
TEST = True

# Functions
def xyz2npmol(xyzfile, return_atomlist=False):
    """Builds NumPy molecule read from XYZ file `xyzfile`.
    
    Parameters
    ----------
    xyzfile : <str>
        Name of file containing Cartesian geometry
    return_atomlist : <bool> (optional, default False)
        If true, returns list of atomic symbols defining molecular geometry
    
    Returns
    -------
    npmol : <ndarray> of <float64>
        (natom, 3) NumPy array of Cartesian molecular geometry
    atomlist : <list> of <str> (optional)
        (natom, ) list of atomic symbols corresponding to molecular geometry
    """
    # Read XYZ file, 
    with open(xyzfile, 'r') as f:
        lines = f.readlines()[2:]
    
    atomlist = []
    npmol = np.zeros([len(lines), 3])
    for l in range(len(lines)):
        npmol[l, :] = np.array([float(i) for i in lines[l].split()[1:]])
        atomlist.append(lines[l].split()[0])
        
    if return_atomlist:
        return npmol, atomlist
    else:
        return npmol
    
def npmol2xyz(npmol, atomlist, xyzfile, comments=None):
    """Void function to write molecular geometry to `xyzfile` corresponding to 
    NumPy molecule `numpymol`.
    
    Arguments
    ---------
    npmol : <ndarray> of <float64>
        (natom, 3) NumPy array of Cartesian molecular geometry
    atomlist : <list> of <str> (optional)
        (natom, ) list of atomic symbols corresponding to molecular geometry
    xyzfile : <str>
        Name of file in which to write Cartesian geometry
    comments : <str> (optional, default None)
        Comments to be written to the second line of XYZ file
    """
    # Build molstr to write to file
    natom = npmol.shape[0]
    molstr = """"""
    molstr += str(natom) + '\n'
    if comments is not None:
        molstr += comments + '\n'
    else:
        molstr += '\n'
    for i in range(natom):
        molstr += atomlist[i] + '\t'
        molstr += str(npmol[i,0]) + '\t'
        molstr += str(npmol[i,1]) + '\t'
        molstr += str(npmol[i,2]) + '\n'
    # Write to xyzfile
    with open(xyzfile, 'w') as xyz:
        xyz.write(molstr)
    
def rmsd(P, Q):
    """Computes the element-wise RMSD between two matrices, P and Q.
    
    Arguments
    ---------
    P : <ndarray> of <float64>
        (3, natom) array of coordinates
    Q : <ndarray> of <float64>
        (3, natom) array of coordinates
    
    Returns
    -------
    r : <float64>
        RMSD between arrays P and Q
    """
    return np.linalg.norm(Q - P) / np.sqrt(P.shape[0])
    
def kabsch_svd(P, Q):
    """Computes the optimal rotation matrix R which maps a set of N points
    P = {p_n | p in R^M} onto a set of N points Q = {q_n | q in R^M} according
    to the minimization of ||Q - P*R||, using the SVD formulation of the Kabsch
    algorithm.

    Arguments
    ---------
    P : <ndarray> of <float64>
        (3, natom) array of concern, with centroid coinciding with the spatial origin (0,0,0)
    Q : <ndarray> of <float64>
        (3, natom) target array, with centroid coinciding with the spatial origin (0,0,0)

    Returns
    -------
    R : <ndarray> of <float64>
        (3,3) optimal rotation matrix mapping P onto Q
    """
    # Form covariance matrix
    cov = Q.dot(P.T)

    # Compute the SVD of the covariance matrix
    U, D, V = np.linalg.svd(cov)

    # Build optimal rotation matrix
    if (np.linalg.det(U) * np.linalg.det(V)) < 0: # Right hand coordinate system?
        U[:, -1] = -U[:, -1]
    R = U.dot(V)

    return R.T
    
def kabsch_quaternion(P, Q):
    """Computes the optimal rotation matrix R which maps a set of N points
    P = {p_n | p in R^M} onto a set of N points Q = {q_n | q in R^M} according
    to the minimization of ||Q - P*R||, using the unit quaterion formulation of
    the Kabsch algorithm.
    
    Arguments
    ---------
    P : <ndarray> of <float64>
        (3, natom) array of concern, with centroid coinciding with the spatial origin (0,0,0)
    Q : <ndarray> of <float64>
        (3, natom) target array, with centroid coinciding with the spatial origin (0,0,0)

    Returns
    -------
    R : <ndarray> of <float64>
        (3,3) optimal rotation matrix mapping P onto Q
    """
    # Form covariance matrix
    cov = Q.dot(P.T)

    # Form the quaternion transformation matrix F
    F = np.zeros((4, 4))
    # diagonal
    F[0, 0] = cov[0, 0] + cov[1, 1] + cov[2, 2]
    F[1, 1] = cov[0, 0] - cov[1, 1] - cov[2, 2]
    F[2, 2] = -cov[0, 0] + cov[1, 1] - cov[2, 2]
    F[3, 3] = -cov[0, 0] - cov[1, 1] + cov[2, 2]
    # Upper & lower triangle
    F[1, 0] = F[0, 1] = cov[1, 2] - cov[2, 1]
    F[2, 0] = F[0, 2] = cov[2, 0] - cov[0, 2]
    F[3, 0] = F[0, 3] = cov[0, 1] - cov[1, 0]
    F[2, 1] = F[1, 2] = cov[0, 1] + cov[1, 0]
    F[3, 1] = F[1, 3] = cov[0, 2] + cov[2, 0]
    F[3, 2] = F[2, 3] = cov[1, 2] + cov[2, 1]

    # Compute ew, ev of F
    ew, ev = np.linalg.eigh(F)

    # Construct optimal rotation matrix from leading ev
    q = ev[:, -1]
    U = np.zeros((3, 3))

    U[0, 0] = q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2
    U[0, 1] = 2 * (q[1] * q[2] - q[0] * q[3])
    U[0, 2] = 2 * (q[1] * q[3] + q[0] * q[2])
    U[1, 0] = 2 * (q[1] * q[2] + q[0] * q[3])
    U[1, 1] = q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2
    U[1, 2] = 2 * (q[2] * q[3] - q[0] * q[1])
    U[2, 0] = 2 * (q[1] * q[3] - q[0] * q[2])
    U[2, 1] = 2 * (q[2] * q[3] + q[0] * q[1])
    U[2, 2] = q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2

    return U
    
def align_molecules(mol, ref, return_aligned=False, alg='svd'):
    """Compares geometry of molecule `mol` to reference geometry `ref`
    using the Kabsch algorithm to compute the optimal alignment of two
    molecular geometries P and Q by minimizing the norm of the residual,
    || Q - P*R||.

    Arguments
    ----------
    mol : <ndarray> of <float64>
        (natom, 3) array of concern/changeable geometry. Assumed [A] for RMSD
        purposes. Must have same Natom, units, and 1-to-1 atom ordering as `ref`.
    ref : <ndarray> of <float64>
        (natom, 3) array of reference/target/unchanged geometry. Assumed [A]
        for RMSD purposes.
    return_aligned : <bool> (optional, default False)
        Return aligned copy of `mol` superimposed on `ref`, both centered at their
        respective centroids.

    Returns
    -------
    rmsd : <float64> 
        Least root mean square displacement between two molecules.
        
    a_mol : <ndarray> of <float64> (optional)
        (natom, 3) array of coordinates for rotated/centered molecule `mol`, 
        superimposed with `ref`
    a_ref : <ndarray> of <float64> (optional)
        (natom, 3) array of coordinates for centered molecule `ref`
    """
    # Translate P & Q to global origin by subtracting away respective centroids
    P = (mol - mol.mean(axis=0))
    Q = (ref - ref.mean(axis=0))
    
    # Compute rotation, rotate P onto Q
    if alg.lower() == 'svd':
        R = kabsch_svd(P.T, Q.T)
    elif alg.lower() == 'quaternion':
        R = kabsch_quaternion(P.T, Q.T)
    elif alg.lower() == 'test':
        return np.allclose(kabsch_svd(P.T, Q.T), kabsch_quaternion(P.T, Q.T))
        
    P = P.dot(R)

    # Compute & Return RMSD
    r = rmsd(P,Q)

    # Return
    if return_aligned:
        return r, P, Q
    else:
        return r
    
def compare_values(test, ref, label):
    """Compares computed values to reference for testing, prints status.
    Raises AssertionError if values do not compare to within 10^-8.
    
    Arguments
    ---------
    test : <float64>
        Computed value to compare to reference.
    ref : <float64>
        Accepted value to compare `test` against.
    label : <str> 
        Labels test. Used in printing of results.
    """
    comparison = np.allclose([test],[ref])
    width = 80
    dots = '.' * (80 - len('        %sPASSED' % label))
    if comparison:
        print('        %s%sPASSED' % (label, dots))
    else:
        print('        %s%sFAILED' % (label, dots))
        assert comparison, 'Test value %8.6f does not match reference value %8.6f' % (test, ref)

In [ ]:
# ==> Example 0: Getting help with functions <==
help(align_molecules)

In [ ]:
# ==> Example 1: Flat Methane <==
# Build NumPy molecules for two flat methanes, aligned along different axes
xy = xyz2npmol('example-geoms/meth_xy.xyz')
yz = xyz2npmol('example-geoms/meth_yz.xyz')

# Compute RMSD before alignment
unaligned_rmsd = rmsd(xy, yz)
aligned_rmsd_s = align_molecules(xy, yz, alg='svd')
aligned_rmsd_q, a_xy, a_yz = align_molecules(xy, yz, alg='quaternion', return_aligned=True)

# Check computed RMSD's

print('RMSD before alignment:            %16.10f [A]' % unaligned_rmsd)
compare_values(unaligned_rmsd, 1.6124515497, "RMSD before alignment")
print('Least RMSD (after SVD alignment): %16.10f [A]' % aligned_rmsd_s)
compare_values(aligned_rmsd_s, 0.4472135955, "LRMSD after SVD alignment")
print('Least RMSD (after Q alignment):   %16.10f [A]' % aligned_rmsd_q)
compare_values(aligned_rmsd_q, 0.4472135955, "LRMSD after quaternion alignment")

# ==> Visualize Before & After Alignment <==
fig = plt.figure(figsize=(20,10))
before = fig.add_subplot(121, projection='3d')
after = fig.add_subplot(122, projection='3d')

before.scatter(xy[:,0], xy[:,1], xy[:,2])
before.scatter(yz[:,0], yz[:,1], yz[:,2])
after.scatter(a_xy[:,0], a_xy[:,1], a_xy[:,2])
after.scatter(a_yz[:,0], a_yz[:,1], a_yz[:,2])

# Plot Options
before.set_title('Before alignment: RMSD = %6.4f [A]' % unaligned_rmsd)
after.set_title('After alignment: LRMSD = %6.4f [A]' % aligned_rmsd_s)
after.set_xlim(-1,1)

In [ ]:
# ==> Example 2: Water Dimer (A24-2) <==
# Build NumPy molecules for B3LYP-D3/DZ water dimer & reference
h2o_dimer = xyz2npmol('example-geoms/A24-2_b3lyp-d3_cc-pvdz_rotated.xyz')
reference = xyz2npmol('example-geoms/A24-2-ref.xyz')

# Compute RMSD before alignment
unaligned_rmsd = rmsd(h2o_dimer, reference)
aligned_rmsd_s = align_molecules(h2o_dimer, reference, alg='svd')
aligned_rmsd_q, a_h2o_dimer, a_reference = align_molecules(h2o_dimer, reference, alg='quaternion',
                                                           return_aligned=True)

# Check computed RMSD's
print('RMSD before alignment:            %16.10f [A]' % unaligned_rmsd)
compare_values(unaligned_rmsd, 1.6603623827, "RMSD before alignment")
print('Least RMSD (after SVD alignment): %16.10f [A]' % aligned_rmsd_s)
compare_values(aligned_rmsd_s, 0.0988999650, "LRMSD after SVD alignment")
print('Least RMSD (after Q alignment):   %16.10f [A]' % aligned_rmsd_q)
compare_values(aligned_rmsd_q, 0.0988999650, "LRMSD after quaternion alignment")

# ==> Visualize Before & After Alignment <==
fig = plt.figure(figsize=(20,10))
before = fig.add_subplot(121, projection='3d')
after = fig.add_subplot(122, projection='3d')

# Plot molecules
before.scatter(h2o_dimer[:,0], h2o_dimer[:,1], h2o_dimer[:,2], color='r', label='B3LYP-D3/DZ')
before.scatter(reference[:,0], reference[:,1], reference[:,2], color='k', label='Reference')
after.scatter(a_h2o_dimer[:,0], a_h2o_dimer[:,1], a_h2o_dimer[:,2], color='r', label='B3LYP-D3/DZ')
after.scatter(a_reference[:,0], a_reference[:,1], a_reference[:,2], color='k', label='Reference')

# Plot Options
before.set_title('Before alignment: RMSD = %6.4f [A]' % unaligned_rmsd)
after.set_title('After alignment: LRMSD = %6.4f [A]' % aligned_rmsd_s)
after.set_xlim(-1,1)
plt.legend(loc='best')